# 크로스 인코더 기반 리랭킹
- 크로스 인코더 기반 리랭킹은 주로 BERT와 같은 인코더 기반 언어 모델을 이용해서 질문과 문서가 얼마나 잘 맞는지 관련성을 평가하는 기법입니다.
- 질문과 문서를 하나로 결합하여 입력하고, 모델이 최종적으로 두 텍스트가 얼마나 관련 있는가를 단일 점수로 출력합니다.

1. 질문, 문서 입력: 질문과 문서를 "[CLS] 질문 [SEP] 문서 [SEP]" 형태로 결합합니다. 여기서 [CLS]는 문장의 시작을 나타내는 토큰이고 [SEP]는 두 텍스트(질문과 문서)를 구분하는 토큰입니다.
2. 인코딩: 이렇게 결합된 문자열을 BERT와 같은 인코더 모델에 넣어, 각 토큰에 대한 임베딩 벡터를 구합니다.
3. [CLS] 토큰 임베딩 추출: 모델의 최종 층에서 나오는 [CLS] 토큰 임베딩은 전체 입력(질문과 문서)의 맥락을 요약한 정보를 담고 있습니다. 이 임베딩은 질문과 문서 사이의 복잡한 의미적 관계를 반영합니다.
4. 점수 계산: [CLS] 임베딩을 간단한 선형 층(fully-connected layer)이나 다른 분류 층에 통과시켜서, 최종적으로 관련성(relevance) 점수를 산출합니다.

### Bi-Encoder VS Cross-Encoder
- 리랭킹을 다룰 때 자주 등장하는 두 가지 인코더 방식, Bi-Encoder와 Cross Encoder에 대해 살펴보겠습니다.
#### Bi-Encoder
- 밀집 검색에서 사용되는 인코딩 방식으로, 밀집 검색의 기반이 되는 벡터 표현을 생성합니다.
- 질문 A와 문서 B가 있을 때, 이들 각각을 임베딩으로 동일한 크기의 벡터로 만든 뒤, 코사인 유사도와 같은 유사도 분석을 통해 벡터 간의 거리를 수치화하는 방식입니다. 두 벡터 간의 거리가 가깝다면 질문과 문서가 의미적으로 유사하다고 판단하고, 멀다면 관련성이 낮다고 판단합니다.

#### Cross-Encoder
- 질문 A와 문서 B가 있을 때, 이들을 트랜스포머 기반의 인코더 모델에 함께 투입하여, 둘 사이의 관련성을 나타내는 분류 점수를 직접 얻습니다. 이 점수가 높을수록 관련성이 높다고 판단합니다.

- 두 인코더 방식은 속도와 정확도에서 뚜렷한 차이를 보입니다.
- Bi-Encoder: 각 문장을 독립적으로 인코딩합니다. 예를 들어 10만개의 문장이 있다면 10만번의 인코딩 작업만 수행하면 됩니다.
- Cross-Encoder: 모든 가능한 문장 쌍을 동시에 인코딩합니다. 10만개의 문장이 있을 경우, 조합에 의해 약 50억개(4,999,950,000개)의 쌍을 인코딩해야 합니다.
- 이러한 작동 방식의 차이로 인해, 대규모 데이터셋에서 Cross-Encoder는 Bi-Encoder에 비해 현저히 느린 속도를 보입니다.
- 특히 수천 개 이상의 문장을 비교해야 하는 경우 Cross-Encoder의 속도 저하는 더욱 두드러집니다.

##### 정확도 비교
- 일반적으로 Cross-Encoder가 Bi-Encoder에 비해 더 높은 정확도를 보입니다.
- Bi-Encoder: 각 문장을 독립적으로 인코딩하기 때문에, 두 문장 간의 관계를 직접적으로 고려하진 않습니다. 따라서 빠른 처리가 가능하지만, 문장 간의 복잡한 상호작용을 포착하는데 한계가 있을 수 있습니다.
- Cross-Encoder: 두 문장을 동시에 인코딩하여 하나의 임베딩을 생성합니다. 이 과정에서 두 문장 간의 관계를 직접적으로 모델링할 수 있어, 더 정확한 분류나 유사도 평가가 가능합니다.
- 결론적으로 속도면에서는 Bi-Encoder, 정확도 측면에서는 Cross-Encoder가 유리합니다.
- 작업의 성격과 요구사항에 따라 적절한 인코더를 선택해야 합니다.
- 실제로는 두 방식을 조합하여 사용하기도 합니다.
- Bi-Encoder를 사용해 대규모 데이터셋에서 후보군을 빠르게 추려낸후 Cross-Encoder를 사용해 후보군을 재평가하여 최종 순위를 결정하는 방식입니다. 빠른 속도와 정확도사이의 균형을 맞출 수 있습니다.

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = "data/투자설명서.pdf"
loader = PyPDFLoader(file_path)
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = loader.load_and_split(doc_splitter)

In [12]:
from langchain_ollama import OllamaEmbeddings
embedding = OllamaEmbeddings(model="bge-m3")

In [13]:
from langchain_community.vectorstores import FAISS

faiss_store = FAISS.from_documents(docs, embedding)

persist_directory = "data/DB"
faiss_store.save_local(persist_directory)

In [14]:
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

- 크로스 인코더 기반 리랭킹 알고리즘을 생성합니다.
- ms-marco-MiniLM-L-12-v2 모델을 사용합니다.
- Microsoft사가 개발한 MS MARCOMicrosoft MAchine Reading COmprehension 데이터셋을 기반으로 학습되었습니다.

In [15]:
from pydantic import BaseModel, Field
from langchain_core.documents import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from sentence_transformers import CrossEncoder
from langchain_core.retrievers import BaseRetriever
from langchain.chains import RetrievalQA

In [8]:
crossencoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

In [16]:
class Retriever_with_cross_encoder(BaseRetriever):
    vectorstore: Any = Field(description="초기 검색을 위한 벡터 저장소")
    crossencoder: Any = Field(description="재순위화를 위한 크로스 인코더 모델")
    k: int = Field(default=5, description="초기에 검색할 문서 수")
    rerank_top_k: int = Field(default=2, description="재순위화 후 최종적으로 반환할 문서 수")

    class Config:
        arbitrary_types_allowed = True
    
    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 초기 검색
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)

        # 인코더용 쌍 준비
        pairs =[[query, doc.page_content] for doc in initial_docs]

        # 인코더 점수 획득
        scores = self.crossencoder.predict(pairs)

        # 점수별 문서 정렬
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)

        # 상위 재순위화 문서 반환
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

- Retruever_with_cross_encoder 클래스의 주요 역할은 벡터 기반 초기 검색과 크로스 인코더를 이용한 재순위화를 결합하여 더 정확한 문서 검색을 수행하는 것입니다.
1. 초기검색: vectorstore를 사용하여 쿼리와 유사한 k개의 문서를 빠르게 검색합니다. 이는 관련 문서의 후보군을 추려내는 역할을 합니다.
2. 문서-질문 쌍 준비: 검색된 각 문서와 질문을 쌍으로 만듭니다. 이는 크로스 인코더의 입력으로 사용됩니다.
3. 관련성 점수 계산: 크로스 인코더 모델을 사용하여 각 질문-문서 쌍의 관련성 점수를 계산합니다.
4. 정렬: 계산된 점수를 기준으로 문서들을 내림차순으로 정렬합니다. 이를 통해 가장 관련성 높은 문서가 상위에 오도록 합니다.
5. 최종 결과 반환: 정렬된 문서 중 상위 rerank_top_k개의 문서만 선택하여 최종 결과로 반환합니다.

In [17]:
#크로스 인코더 기반 리트리버 인스턴스 생성
cross_encoder_retriever = Retriever_with_cross_encoder(
    vectorstore=vectordb,
    crossencoder=crossencoder,
    k=4,
    rerank_top_k=2
)

# 답변용 LLM 인스턴스 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

- vectorstore설정: 앞서 생성한 vectordb를 벡터 저장소로 지정하여 초기 밀집 검색에 사용합니다.
- crossencoder설정: 미리 준비된 crossencoder 모델을 지정하여 검색된 문서의 재순위화에 사용합니다.
- k값 설정: 초기 밀집 검색에서 반환할 문서의 수를 4로 설정합니다. 이는 벡터 검색을 통해 먼저 4개의 관련 문서를 추출함을 의미합니다.
- rerank_top_k값 설정: 리랭킹 후 최종적으로 반환할 문서의 수를 2로 설정합니다. 즉, 크로스 인코더를 통해 재평가된 문서 중 가장 관련성 높은 2개만을 최종 결과로 선택합니다.

In [18]:
# RetruevalQA 체인 인스턴스 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True,
)

In [19]:
query = "이 회사의 2022년 영업손실이 정확히 얼마야?"
result = qa_chain.invoke({"query": query})

print(f"\n질문: {query}")
print(f"답변: {result['result']}")
print("\n답변 근거 문서:")
for i, doc in enumerate(result['source_documents']):
    print(f"\Document {i+1}:")
    print(doc.page_content)


질문: 이 회사의 2022년 영업손실이 정확히 얼마야?
답변: 2022년 영업손실은 149.1억원입니다.

답변 근거 문서:
\Document 1:
하여 2021년 영업손실 130.1억원, 2022년 영업손실 149.1억원, 2023년 영업손실 122억원, 2024년
1분기 영업손실 24.2억원이 발생하였습니다. 또한 영업 외적 측면에서도, 금융비용 등의 발생 영향
으로 인해 2021년 당기순손실 130.7억원, 2022년 당기순손실 228.7억원, 2023년 당기순손실
116.1억원, 2024년 1분기 당기순손실 32.9억원이 발생하는 등 지속적인 적자 구조를 면하지 못하고
있습니다.따라서 당사의 파이프라인에서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루
\Document 2:
외적 측면에서도, 금융비용 등의 발생 영향으로 인해 2021년 당기순손실 130.7억원, 2022년
당기순손실 228.7억원, 2023년 당기순손실 116.1억원, 2024년 1분기 당기순손실 32.9억원
이 발생하는 등 지속적인 적자 구조를 면하지 못하고 있습니다. 따라서 당사의 파이프라인에
서 임상 성공을 통한 기술이전, 상품화 성공 등의 성과를 이루어내지 못한다면 당사의 적자
구조를 개선하는 것은 불가능할 수 있으며, 자본력이 지속적으로 감소하여 지속적인 연구개


- 크로스 인코더 기반 리랭킹과 고성능 언어 모델 기반 리랭킹의 차이가 있습니다.
- 크로스 인코더는 인코더 기반 트랜스포머를 사용하여 고성능 대규모 언어 모델에 비해 빠른 처리 속도를 제공합니다.
- 그러나 정확도 측면에서는 고성능 언어 모델 기반에 비해 다소 떨어질 수 있습니다.

- 작업의 특성과 요구사항을 고려하여 빠른 처리가 필요할 경우 크로스 인코더 기반 방식, 높은 정확도가 필요할 경우 고성능 언어 모델 기반 방식이 더 적합할 수 있습니다.